In [ ]:
def returns(
        contributions: 'pd',
        balance: 'pd',
        col_contrb: str,
        col_balance: str,
        col_sub: str,
        col_ratio: str
        ) -> 'pd':
    # Merge two files
    returns = contributions.merge(balance,left_index=True,right_index=True)
    # Add Column with the subtraction of column2 from column1
    returns[col_sub] = (returns[col_balance] - returns[col_contrb])
    # Save as Integer
    returns[col_sub] = (returns[col_sub].astype('int'))
    # Add Column with the ratio of column2 from column1
    returns[col_ratio] = ((returns[col_balance] / returns[col_contrb])-1)
    # Drop individual columns
    returns.drop([col_contrb, col_balance], axis=1,inplace=True)
    # Return dataframe
    return returns

In [1]:
import pandas as pd
from scripts import daily_balance as db

In [2]:


col_sub = 'Tot_Portfolio_Return_MXN'
col_ratio='Tot_Portfolio_Return_Percent'

In [3]:
# Merge two files
def merge_df(contributions, balance):
    merged_file = contributions.merge(balance,left_index=True,right_index=True)
    return merged_file

contributions = db.create_df('outputs/daily_contributions_CLG_AllAccounts.csv')
balance = db.create_df('outputs/daily_acct_balance_CLG_AllAccounts.csv')
merged = merge_df(contributions, balance)
merged

Date,Contribuciones_GBM_MXN,Contribuciones_Cetes_MXN,Tot_Contribuciones_MXN,Tot_Acct_Cetes_MXN,Tot_Acct_GBM_MXN,Tot_Acct_Portafolio_MXN
2022-08-15,5600000,3158672,8758672,3470415,5094902,8565317
2022-08-14,5600000,3158672,8758672,3470415,5094902,8565317
2022-08-13,5600000,3158672,8758672,3470415,5094902,8565317
2022-08-12,5600000,3158672,8758672,3470415,5094902,8565317
2022-08-11,5600000,3158672,8758672,3470415,5094902,8565317
...,...,...,...,...,...,...
2019-12-17,1500000,816738,2316738,2672321,1000525,3672846
2019-12-16,1500000,816738,2316738,2672321,1000525,3672846
2019-12-15,1500000,816738,2316738,2672321,1000525,3672846
2019-12-14,1500000,816738,2316738,2672321,1000525,3672846


In [4]:
# Create a column with the subtraction of column2 from column1
def calc_returns(df, col_balance, col_contrb):
    df[col_sub] = (df[col_balance] - df[col_contrb])
    return df

col_contrb = 'Tot_Contribuciones_MXN'
col_balance = 'Tot_Acct_Portafolio_MXN'
port_return = calc_returns(merged, col_balance, col_contrb)
port_return

Date,Contribuciones_GBM_MXN,Contribuciones_Cetes_MXN,Tot_Contribuciones_MXN,Tot_Acct_Cetes_MXN,Tot_Acct_GBM_MXN,Tot_Acct_Portafolio_MXN,Tot_Portfolio_Return_MXN
2022-08-15,5600000,3158672,8758672,3470415,5094902,8565317,-193355
2022-08-14,5600000,3158672,8758672,3470415,5094902,8565317,-193355
2022-08-13,5600000,3158672,8758672,3470415,5094902,8565317,-193355
2022-08-12,5600000,3158672,8758672,3470415,5094902,8565317,-193355
2022-08-11,5600000,3158672,8758672,3470415,5094902,8565317,-193355
...,...,...,...,...,...,...,...
2019-12-17,1500000,816738,2316738,2672321,1000525,3672846,1356108
2019-12-16,1500000,816738,2316738,2672321,1000525,3672846,1356108
2019-12-15,1500000,816738,2316738,2672321,1000525,3672846,1356108
2019-12-14,1500000,816738,2316738,2672321,1000525,3672846,1356108


In [5]:
    # Save as Integer
    def df_to_int(df, column):
        df[column] = (df[column].astype('int'))
        return df
    
    port_return = df_to_int(port_return, col_sub)
    port_return

Date,Contribuciones_GBM_MXN,Contribuciones_Cetes_MXN,Tot_Contribuciones_MXN,Tot_Acct_Cetes_MXN,Tot_Acct_GBM_MXN,Tot_Acct_Portafolio_MXN,Tot_Portfolio_Return_MXN
2022-08-15,5600000,3158672,8758672,3470415,5094902,8565317,-193355
2022-08-14,5600000,3158672,8758672,3470415,5094902,8565317,-193355
2022-08-13,5600000,3158672,8758672,3470415,5094902,8565317,-193355
2022-08-12,5600000,3158672,8758672,3470415,5094902,8565317,-193355
2022-08-11,5600000,3158672,8758672,3470415,5094902,8565317,-193355
...,...,...,...,...,...,...,...
2019-12-17,1500000,816738,2316738,2672321,1000525,3672846,1356108
2019-12-16,1500000,816738,2316738,2672321,1000525,3672846,1356108
2019-12-15,1500000,816738,2316738,2672321,1000525,3672846,1356108
2019-12-14,1500000,816738,2316738,2672321,1000525,3672846,1356108


In [6]:
# Create column with the ratio of column2 from column1
def ratio(df, col_ratio, col_balance, col_contrb):
    df[col_ratio] = ((df[col_balance] / df[col_contrb])-1)
    return df

port_return = ratio(port_return, col_ratio, col_balance, col_contrb)
port_return

Date,Contribuciones_GBM_MXN,Contribuciones_Cetes_MXN,Tot_Contribuciones_MXN,Tot_Acct_Cetes_MXN,Tot_Acct_GBM_MXN,Tot_Acct_Portafolio_MXN,Tot_Portfolio_Return_MXN,Tot_Portfolio_Return_Percent
2022-08-15,5600000,3158672,8758672,3470415,5094902,8565317,-193355,-0.022076
2022-08-14,5600000,3158672,8758672,3470415,5094902,8565317,-193355,-0.022076
2022-08-13,5600000,3158672,8758672,3470415,5094902,8565317,-193355,-0.022076
2022-08-12,5600000,3158672,8758672,3470415,5094902,8565317,-193355,-0.022076
2022-08-11,5600000,3158672,8758672,3470415,5094902,8565317,-193355,-0.022076
...,...,...,...,...,...,...,...,...
2019-12-17,1500000,816738,2316738,2672321,1000525,3672846,1356108,0.585352
2019-12-16,1500000,816738,2316738,2672321,1000525,3672846,1356108,0.585352
2019-12-15,1500000,816738,2316738,2672321,1000525,3672846,1356108,0.585352
2019-12-14,1500000,816738,2316738,2672321,1000525,3672846,1356108,0.585352


In [13]:
# Drop individual columns
def drop_column(df, *args):
    df.drop(list(args), axis=1,inplace=True)
    return df

port_return = drop_column(port_return, col_contrb, col_balance, )
port_return


Date,Contribuciones_GBM_MXN,Contribuciones_Cetes_MXN,Tot_Acct_Cetes_MXN,Tot_Acct_GBM_MXN,Tot_Portfolio_Return_MXN,Tot_Portfolio_Return_Percent
2022-08-15,5600000,3158672,3470415,5094902,-193355,-0.022076
2022-08-14,5600000,3158672,3470415,5094902,-193355,-0.022076
2022-08-13,5600000,3158672,3470415,5094902,-193355,-0.022076
2022-08-12,5600000,3158672,3470415,5094902,-193355,-0.022076
2022-08-11,5600000,3158672,3470415,5094902,-193355,-0.022076
...,...,...,...,...,...,...
2019-12-17,1500000,816738,2672321,1000525,1356108,0.585352
2019-12-16,1500000,816738,2672321,1000525,1356108,0.585352
2019-12-15,1500000,816738,2672321,1000525,1356108,0.585352
2019-12-14,1500000,816738,2672321,1000525,1356108,0.585352
